In [5]:
import pandas as pd
df_t = pd.read_csv('transcript_all.csv')
df_c = df_t.groupby('chat_id').text.apply(' '.join)
df_topic = pd.read_csv('df_topic.csv')
top = pd.read_csv('top.csv')

In [6]:
df_topic['chat_id'] = df_c.index
qa = pd.read_csv('QApairs.csv')
df_qa = pd.merge(qa, df_topic.loc[:,['topic_no','chat_id','keywords']], on='chat_id').fillna(' ')
df_q = pd.DataFrame(df_qa.groupby(['chat_id','topic_no'])['query'].apply(' '.join)).reset_index()
df_q = df_q.loc[df_q.apply(lambda s: not s['query'].isspace(), axis=1)]

In [170]:
slist.loc[slist.feature == 'king']

,feature,importance
104,king,0.002878


In [292]:
slist = pd.read_csv('stop_words_product.csv')[4:120]
# Tokenize chats
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['help', 'headout', 'hi', 'hello', 'hey', 'welcome', 'stepping', 'away', 'customers', 'bye', 'problem', 'feel', 'free', 'reach', 'need', 'assistance', 'would', 'nice', 'talk', 'great', 'day', 'goodbye', 'give', 'minutes', 'like', 'assist', 'anything', 'else', 'today', 'thank', 'check', 'please', 'ok','get', 'yes', 'safari', 'khalifa', 'burj', 'aquarium', 'roman', 'palatin', 'vatican', 'dubai'])
stop_words.extend(slist.feature.values.tolist())
def remove_stopwords(l):
    return [word for word in l if word not in stop_words]

from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

porter_stemmer = PorterStemmer()
def stem_words(l):
    return [porter_stemmer.stem(words) for words in l]

wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize_words(l):
    return [wordnet_lemmatizer.lemmatize(words) for words in l]
def nget(l):
    return [word for word in l if word!='get']
def preprocess(text):
    return nget(lemmatize_words(stem_words(remove_stopwords(tokenizer.tokenize(text.lower())))))
preprocess('I will be stepping away get to assist other customers now, bye! :)')

[]

In [269]:
'get' in stop_words

True

In [293]:
q = df_q['query'].apply(lambda s: ' '.join(preprocess(s)))

In [294]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(ngram_range=(1,2),max_df=500, min_df=10)

x = tv.fit_transform(q)
y = df_q.topic_no

In [272]:
terms = pd.DataFrame({'term': tv.get_feature_names(), 'tfidf': x.toarray().sum(0)})
terms.sort_values('tfidf',ascending=False)

,term,tfidf
1122,seat,110.154855
1180,show,86.209427
190,avail,82.440411
1141,see,75.957324
1033,purchas,74.294109
778,look,71.651646
1008,price,64.811428
285,buy ticket,64.470892
417,date,62.471393
258,book ticket,61.268422


In [295]:
'get' in tv.get_feature_names()

False

In [274]:
terms.loc[terms.apply(lambda s: s.term == 'get',1)]

,term,tfidf
599,get,2.61941


In [253]:
terms.sort_values('tfidf',ascending=False)[200:]

,term,tfidf
429,de,15.988901
1248,sunday,15.837923
746,later,15.696497
761,link,15.558814
29,16,15.545433
1504,want buy,15.419106
822,mean,15.410611
1502,want ask,15.371298
1029,promo code,15.345556
583,friday,15.341991


In [200]:
x

<4354x1580 sparse matrix of type '<class 'numpy.float64'>'
	with 68534 stored elements in Compressed Sparse Row format>

In [275]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [296]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
sgd = SGDClassifier(random_state=100)
sgd_param = {'alpha': [0.001, 0.01, 0.0001, 0.1]
            }
clf = GridSearchCV(sgd, sgd_param, cv=5, n_jobs=-1, scoring='f1_macro')

In [234]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb_param = {'alpha': [0.2, 0.1, 0.3, 0.005]}
clf = GridSearchCV(nb, nb_param, cv=5, n_jobs=-1, scoring='f1_macro')

In [297]:
%%time
clf.fit(x_train, y_train)

CPU times: user 151 ms, sys: 32.1 ms, total: 183 ms
Wall time: 566 ms


GridSearchCV(cv=5, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=100, shuffle=True,
       tol=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.001, 0.01, 0.0001, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=0)

In [298]:
clf.cv_results_

{'mean_fit_time': array([0.04042749, 0.03349261, 0.02267499, 0.02835269]),
 'std_fit_time': array([0.0007586 , 0.00646512, 0.00222772, 0.0011291 ]),
 'mean_score_time': array([0.0023912 , 0.00176005, 0.00112548, 0.00110478]),
 'std_score_time': array([2.90083900e-04, 5.41371527e-04, 4.20246410e-05, 1.27104377e-04]),
 'param_alpha': masked_array(data=[0.001, 0.01, 0.0001, 0.1],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.001},
  {'alpha': 0.01},
  {'alpha': 0.0001},
  {'alpha': 0.1}],
 'split0_test_score': array([0.40851315, 0.41526016, 0.41890948, 0.11277862]),
 'split1_test_score': array([0.41874406, 0.40685658, 0.4486862 , 0.10639493]),
 'split2_test_score': array([0.42868386, 0.41138803, 0.43295679, 0.12658532]),
 'split3_test_score': array([0.40368159, 0.40385401, 0.42578057, 0.0675746 ]),
 'split4_test_score': array([0.42855453, 0.42479922, 0.43657331, 0.09559452]),
 'mean_test_score': array([0.417604

In [140]:
clf.cv_results_

{'mean_fit_time': array([0.04327898, 0.03229651]),
 'std_fit_time': array([0.00335704, 0.00733333]),
 'mean_score_time': array([0.0023983, 0.0014771]),
 'std_score_time': array([0.00022847, 0.00039909]),
 'param_alpha': masked_array(data=[0.001, 0.002],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.001}, {'alpha': 0.002}],
 'split0_test_score': array([0.44638971, 0.43499331]),
 'split1_test_score': array([0.48368509, 0.47151603]),
 'split2_test_score': array([0.47971297, 0.46926419]),
 'split3_test_score': array([0.45816959, 0.4534119 ]),
 'split4_test_score': array([0.5105392 , 0.49644419]),
 'mean_test_score': array([0.47558307, 0.46501052]),
 'std_test_score': array([0.02214565, 0.02039563]),
 'rank_test_score': array([1, 2], dtype=int32),
 'split0_train_score': array([0.81131243, 0.78783472]),
 'split1_train_score': array([0.81236261, 0.79685381]),
 'split2_train_score': array([0.81059502, 0.79173531]),
 'split3_train

In [141]:
sgd = SGDClassifier(alpha=0.001, random_state=100, n_jobs=-1)
sgd.fit(x_train,y_train)

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=100,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [142]:
f1_score(sgd.predict(x_test), y_test, average='weighted')

0.5519244610708467

In [64]:
pd.Series(sgd.decision_function(tv.transform(['what about this lion king']))[0]).sort_values()

2    -1.693015
11   -1.594039
3    -1.578137
15   -1.534652
5    -1.513845
0    -1.442275
19   -1.427674
17   -1.424160
14   -1.319785
18   -1.307455
10   -1.300718
4    -1.205769
7    -1.202773
13   -1.163881
8    -1.163825
16   -1.133505
9    -1.068774
12   -1.066628
1    -0.884290
6    -0.572425
dtype: float64

In [299]:
sgd = SGDClassifier(alpha=0.0001, random_state=100, n_jobs=-1)
sgd.fit(x,y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=100,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [237]:
nb = MultinomialNB(alpha=0.1)
nb.fit(x,y)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [217]:
nb.score(x,y)

0.8240698208543867

In [280]:
sgd.score(x,y)

0.8713826366559485

In [304]:
import joblib

# save tfidf
filename = 'Query_Vectorizer.sav'
joblib.dump(tv, filename)

# save sgd model
filename = 'Query_Model.sav'
joblib.dump(sgd, filename)

['Query_Model.sav']

In [54]:
top

,topic_no,topic_name
0,2,time
1,5,reservation
2,6,discount offer
3,9,select seat
4,10,refund
5,11,children
6,14,book a tour
7,15,ticket
8,18,coupon code
9,19,cashback


In [301]:
sgd.predict(tv.transform(['is there any discount for lion king']))

array([6.])

In [300]:
sgd.predict(tv.transform(['i  would like to get a refund for opeara']))

array([10.])

In [312]:
sgd.predict(tv.transform(['can i  choose seat n for opeara']))

array([9.])

In [318]:
sgd.decision_function(tv.transform(['is there any discount for lion king'])).max()

1.7979626191832827